<a href="https://colab.research.google.com/github/sourcecode369/transformers-tutorials/blob/master/rnn/TextRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [37]:
sentences = ["i like nlp", "i love gini", "i eat lot", "i heart chicken"]
word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)
batch_size = len(sentences)

In [38]:
n_step = 2
n_hidden = 5

In [39]:
class TextRNN(nn.Module):
    def __init__(self):
        super(TextRNN, self).__init__()
        self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden)
        self.W = nn.Linear(n_hidden, n_class)
        self.b = nn.Parameter(torch.ones([n_class]))
    
    def forward(self, hidden, X):
        X = X.transpose(0, 1) # X : [n_step, batch_size, n_class]
        outputs, hidden = self.rnn(X, hidden)
        # outputs : [n_step, batch_size, num_directions(=1) * n_hidden]
        # hidden : [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        outputs = outputs[-1]
        model = self.W(outputs) + self.b
        return model

In [40]:
model = TextRNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [41]:
def make_batch():
    input_batch = []
    target_batch = []
    for sen in sentences:
        word = sen.split()  # space tokenizer
        input = [word_dict[n] for n in word[:-1]]  # create (1~n-1) as input
        target = word_dict[word[-1]]  # create (n) as target, We usually call this 'casual language model'
        
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)
    return input_batch, target_batch

In [42]:
input_batch, target_batch = make_batch()
input_batch = torch.FloatTensor(input_batch)
target_batch = torch.LongTensor(target_batch)

In [43]:
for epoch in range(5000):
    optimizer.zero_grad()
    # hidden : [num_layers * num_directions, batch, hidden_size]
    hidden = torch.zeros(1, batch_size, n_hidden)
    # input_batch : [batch_size, n_step, n_class]
    output = model(hidden, input_batch)
    # output : [batch_size, n_class], target_batch : [batch_size] (LongTensor, not one-hot)
    loss = criterion(output, target_batch)
    if (epoch + 1) % 500 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

Epoch: 0500 cost = 0.613180
Epoch: 1000 cost = 0.153365
Epoch: 1500 cost = 0.053604
Epoch: 2000 cost = 0.025817
Epoch: 2500 cost = 0.015055
Epoch: 3000 cost = 0.009713
Epoch: 3500 cost = 0.006650
Epoch: 4000 cost = 0.004728
Epoch: 4500 cost = 0.003449
Epoch: 5000 cost = 0.002560


In [44]:
input = [sen.split()[:2] for sen in sentences]
hidden = torch.zeros(1, batch_size, n_hidden)
predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

[['i', 'like'], ['i', 'love'], ['i', 'eat'], ['i', 'heart']] -> ['nlp', 'gini', 'lot', 'chicken']
